In [16]:
import torch
import torch.nn as nn

from functools import partial
from model import Transformer, ModelArgs
from lora import add_lora, remove_lora

%autoreload 2

UsageError: Line magic function `%autoreload` not found.


## model

In [17]:
# stories 42M
dim = 512
n_layers = 8
n_heads = 8
n_kv_heads = 8
multiple_of = 32
dropout = 0.0
vocab_size = 32000
max_seq_len = 1024

model_args = dict(
    dim=dim,
    n_layers=n_layers,
    n_heads=n_heads,
    n_kv_heads=n_kv_heads,
    vocab_size=vocab_size,
    multiple_of=multiple_of,
    max_seq_len=max_seq_len,
    dropout=dropout,
)

gptconf = ModelArgs(**model_args)
model = Transformer(gptconf)

print(model)

Transformer(
  (tok_embeddings): Embedding(32000, 512)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-7): 8 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=512, out_features=512, bias=False)
        (wk): Linear(in_features=512, out_features=512, bias=False)
        (wv): Linear(in_features=512, out_features=512, bias=False)
        (wo): Linear(in_features=512, out_features=512, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=512, out_features=1376, bias=False)
        (w2): Linear(in_features=1376, out_features=512, bias=False)
        (w3): Linear(in_features=512, out_features=1376, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=512,

In [20]:
lora_rank = 8
lora_alpha = 16
lora_dropout_p = 0.05
target_modules = ['wq', 'wk']

remove_lora(model)
add_lora(model, rank=lora_rank, alpha=lora_alpha, dropout_p=lora_dropout_p, target_modules=target_modules)
print(model)
# register_lora_layer(model, lora_rank, lora_alpha, lora_dropout_p, target_modules)
# model.apply(partial(register_lora_layer, lora_rank=lora_rank, lora_alpha=lora_alpha, lora_dropout_p=lora_dropout_p, target_modules=target_modules))

add lora to layers.0.attention.wq
add lora to layers.0.attention.wk
add lora to layers.1.attention.wq
add lora to layers.1.attention.wk
add lora to layers.2.attention.wq
add lora to layers.2.attention.wk
add lora to layers.3.attention.wq
add lora to layers.3.attention.wk
add lora to layers.4.attention.wq
add lora to layers.4.attention.wk
add lora to layers.5.attention.wq
add lora to layers.5.attention.wk
add lora to layers.6.attention.wq
add lora to layers.6.attention.wk
add lora to layers.7.attention.wq
add lora to layers.7.attention.wk
Transformer(
  (tok_embeddings): Embedding(32000, 512)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-7): 8 x TransformerBlock(
      (attention): Attention(
        (wq): ParametrizedLinear(
          in_features=512, out_features=512, bias=False
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): LoRA(in_features=512, out_features=512, weight_type=linear, lora_rank=8, lora

In [9]:
def get_lora_params(model, print_shapes=True):
    def name_is_lora(name):
        return (
            len(name.split(".")) >= 4
            and (name.split(".")[-4]) == "parametrizations"
            and name.split(".")[-1] in ["lora_A", "lora_B"]
        )
    for n, p in model.named_parameters():
        if name_is_lora(n):
            if print_shapes:
                print(n, p.shape)
            yield p


In [10]:
for n in get_lora_params(model):
    continue

layers.0.attention.wq.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.0.attention.wq.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.0.attention.wk.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.0.attention.wk.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.1.attention.wq.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.1.attention.wq.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.1.attention.wk.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.1.attention.wk.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.2.attention.wq.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.2.attention.wq.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.2.attention.wk.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.2.attention.wk.parametrizations.weight.0.lora_B torch.Size([8, 512])
layers.3.attention.wq.parametrizations.weight.0.lora_A torch.Size([512, 8])
layers.3.att

In [21]:
ckpt = torch.load("./out/tinyshakespeare_lora_stories260k_default_nodropout_0822_1536/ckpt.pt")

In [24]:
ckpt["config"]

{'out_dir': 'out/tinyshakespeare_lora_stories260k_default_nodropout_0822_1536',
 'eval_interval': 2000,
 'log_interval': 1,
 'eval_iters': 100,
 'eval_only': False,
 'always_save_checkpoint': False,
 'init_from': 'pretrained:out/stories260k_default/ckpt.pt',
 'wandb_log': True,
 'wandb_project': 'llamac',
 'wandb_run_name': 'tinyshakespeare_lora_stories260k_default_nodropout_0822_1536',
 'batch_size': 128,
 'max_seq_len': 512,
 'vocab_source': 'custom',
 'vocab_size': 512,
 'dataset': 'tinyshakespeare',
 'dim': 288,
 'n_layers': 6,
 'n_heads': 6,
 'n_kv_heads': 6,
 'multiple_of': 32,
 'dropout': 0.0,
 'use_lora': True,
 'lora_rank': 16,
 'lora_alpha': 1,
 'lora_dropout_p': 0.0,
 'gradient_accumulation_steps': 1,
 'learning_rate': 0.001,
 'max_iters': 20000,
 'weight_decay': 0.01,
 'beta1': 0.9,
 'beta2': 0.99,
 'grad_clip': 1.0,
 'decay_lr': True,
 'warmup_iters': 1000,
 'device': 'cuda',
 'dtype': 'bfloat16',
 'compile': True}

## dataloader

#### inspect the tiny story

In [1]:
import numpy as np
import requests
import torch
import json

In [2]:
m = np.memmap("./data/tok512/data00.bin", dtype=np.uint16, mode="r")

In [3]:
m

memmap([  1, 317, 269, ..., 287, 411, 426], dtype=uint16)

In [4]:
with open("./data/TinyStories_all_data/data00.json", "r") as f:
    data = json.load(f)

In [14]:
print(data[0]['story'])



Lily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.
"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.
Ben watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.
"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.
Lily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.
"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is gone.
Lily feels sorry. She wishes she had shared the swing with Ben. She wishes he was there to hug her. She limps to the tree. She sees something hanging from a branch. It is Ben's hat. He 

In [6]:
from tokenizer import Tokenizer

In [7]:
tok = Tokenizer("./data/tok512.model")

In [10]:
print(tok.decode(m.tolist()[:1000]))

Lily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.
"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.
Ben watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.
"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.
Lily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.
"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is gone.
Lily feels sorry. She wishes she had shared the swing with Ben. She wishes he was there to hug her. She limps to the tree. She sees something hanging from a branch. It is Ben's hat. He le

#### prepare tinyshakespeare

1. encode the whole text
2. save it into a .bin
3. dataloader load it

## register_parametrization

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.utils.parametrize as parametrize
import torch.autograd.profiler as profiler

In [2]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(300, 300)
    
    def forward(self, x):
        out = self.linear(x)
        return out

class LoRA(nn.Module):
    def __init__(self):
        super().__init__()
        self.lora_A = nn.Parameter(torch.zeros(300, 300))
        self.lora_B = nn.Parameter(torch.zeros(300, 300))
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
        self.dropout = nn.Dropout(0.01)
        self.register_buffer(
            "lora_dropout_mask", torch.ones(300, 1, dtype=self.lora_A.dtype)
        )
        
    def forward(self, weight):
        print("foward!")
        dropout_lora_A = self.dropout(self.lora_dropout_mask) * self.lora_A
        delta = dropout_lora_A @ self.lora_B
        return weight + delta.T

In [3]:
torch.cuda.reset_peak_memory_stats(device=None)
model = Model()
model.cuda()
x = torch.randn(1, 300).cuda()
out = model(x)
print(f"gpu used {torch.cuda.max_memory_allocated(device=None)} memory")

gpu used 9933312 memory


In [3]:
torch.cuda.reset_peak_memory_stats(device=None)
model = Model()
parametrize.register_parametrization(model.linear, "weight", LoRA())
model.cuda()
x = torch.randn(1, 300).cuda()
out = model(x)
print(f"gpu used {torch.cuda.max_memory_allocated(device=None)} memory")

foward!
foward!
gpu used 11378176 memory


In [34]:
model = Model()
model.cuda()
parametrize.register_parametrization(model.linear2, "weight", Symmetric())

foward!


ParametrizedLinear(
  in_features=3, out_features=3, bias=True
  (parametrizations): ModuleDict(
    (weight): ParametrizationList(
      (0): Symmetric()
    )
  )
)

In [40]:
model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[ 0.0129, -0.2169,  0.3881],
                      [-0.5198,  0.2688, -0.4700],
                      [ 0.0152,  0.1100,  0.1116]], device='cuda:0')),
             ('linear1.bias',
              tensor([-0.0981,  0.1367,  0.0430], device='cuda:0')),
             ('linear2.bias',
              tensor([0.3150, 0.5501, 0.5367], device='cuda:0')),
             ('linear2.parametrizations.weight.original',
              tensor([[ 0.1634,  0.5439,  0.1278],
                      [ 0.3339,  0.2044, -0.2386],
                      [-0.0448, -0.5470, -0.0414]], device='cuda:0'))])

In [35]:
x = torch.randn(1, 3).cuda()

In [36]:
# warmup
model(x)

with profiler.profile(profile_memory=True, use_cuda=True) as prof:
    out = model(x)
    # out = model(x)

foward!
foward!


STAGE:2023-08-26 15:07:46 9482:9482 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-26 15:07:46 9482:9482 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-26 15:07:46 9482:9482 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [38]:
print(prof.key_averages(group_by_stack_n=5).table())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           LINEAR PASS1        23.75%     351.000us        44.25%     654.000us     654.000us     355.000us        23.91%     668.000us     668.000us           0 b           0 b         512 b           0 

In [31]:
prof.export_chrome_trace("trace.json")

In [14]:
print(f"gpu used {torch.cuda.max_memory_allocated(device=None)} memory")

gpu used 2560 memory
